#### Ingest circuits.csv file

In [0]:
# dbutils.widgets.help()

In [0]:
dbutils.widgets.text("p_data_source","")
v_data_source=dbutils.widgets.get("p_data_source")

In [0]:
dbutils.widgets.text("p_file_date","2021-03-21")
v_file_date=dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

##### Step 1 Read the csv file using the spark dataframe reader


In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, FloatType

In [0]:
circuits_schema=StructType(fields=
                           [StructField("circuitId",IntegerType(),False),
                            StructField("circuitRef",StringType(),True),
                            StructField("name",StringType(),True),
                            StructField("location",StringType(),True),
                            StructField("country",StringType(),True),
                            StructField("lat",DoubleType(),True),
                            StructField("lng",DoubleType(),True),
                            StructField("alt",IntegerType(),True),
                            StructField("url",StringType(),True)
])


In [0]:
circuits_df=spark.read \
    .option("header",True) \
    .schema(circuits_schema) \
    .csv(f"{raw_folder_path}/{v_file_date}/circuits.csv")

In [0]:
#circuits_df.printSchema()

#####Step 2 - Select Only the rquired columns


In [0]:
# we can use the select method to select columns only
circuits_selected_df=circuits_df.select("circuitId","circuitRef","name","location","country","lat","lng","alt")
#display(circuits_selected_df)

In [0]:
# we can use the select method to select columns with df.column_name
circuits_selected_df=circuits_df.select(circuits_df.circuitId,circuits_df.circuitRef,circuits_df.name,circuits_df.location,circuits_df.country,circuits_df.lat,circuits_df.lng,circuits_df.alt)
#display(circuits_selected_df)

In [0]:
#we can use the select method to select columns with df["column_name"]
circuits_selected_df=circuits_df.select(circuits_df["circuitId"],circuits_df["circuitRef"],circuits_df["name"],circuits_df["location"],circuits_df["country"],circuits_df["lat"],circuits_df["lng"],circuits_df["alt"])
#display(circuits_selected_df)

In [0]:
from pyspark.sql.functions import col,lit

In [0]:
#we can use the select method to select columns with col("column_name")
circuits_selected_df=circuits_df.select(col("circuitId"),col("circuitRef"),col("name"),col("location"),col("country"),col("lat"),col("lng"),col("alt"))


#####Step 3 - Renaming the column as required


In [0]:
circuits_renamed_df=circuits_selected_df.withColumnRenamed("circuitId","circuit_id") \
    .withColumnRenamed("circuitRef","circuit_ref") \
    .withColumnRenamed("lat","latitude") \
    .withColumnRenamed("lng","lngitude") \
    .withColumnRenamed("alt","alitude") \
    .withColumn("data_source",lit(v_data_source)) \
    .withColumn("file_date",lit(v_file_date))

#####Step 4 - Add ingestion date

In [0]:
# from pyspark.sql.functions import current_timestamp

In [0]:
# circuits_final_df=circuits_renamed_df.withColumn("ingestion_date",current_timestamp()) \
#     .withColumn("env",lit("dev"))

In [0]:
# circuits_final_df=circuits_renamed_df.withColumn("ingestion_date",current_timestamp())
circuits_final_df=add_ingestion_date(circuits_renamed_df)

#####step 5 - Write data to data lake as parquet


In [0]:
# display(dbutils.fs.ls("/mnt/formuladl15/"))
#/mnt/formuladl15/process/

In [0]:
# circuits_final_df.write.mode("overwrite").parquet(f"{processed_folder_path}/circuits")

In [0]:
# f1_processed
circuits_final_df.write.mode("overwrite").format("delta").saveAsTable("f1_processed.circuits")

In [0]:
# df = spark.read.parquet(f"{processed_folder_path}/circuits")
# display(df)

In [0]:
%sql
select * from f1_processed.circuits;

In [0]:
dbutils.notebook.exit("Success")